In [1]:
import requests
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import time
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

load_dotenv()
SNF_USER = os.getenv('snf_user')
SNF_PASSWORD = os.getenv('snf_password')
SNF_ACCOUNT = os.getenv('snf_account_name')
SNF_WAREHOUSE = os.getenv('snf_warehouse')
SNF_DATABASE = os.getenv('snf_db')
API_KEY = os.getenv('API_KEY')



In [2]:
def fetch_ohlc_data(symbol):
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=6*365)
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&to={end_date}&apikey={API_KEY}"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get('historical', [])
        return pd.DataFrame(data)
    else:
        print(f"Failed to fetch data for {symbol}")
        return pd.DataFrame()


In [3]:
fetch_ohlc_data("AAPL")


,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
0,2024-02-27,181.10,183.9225,179.56,182.63,182.63,53345536,53266403,1.53,0.84484,181.80,"February 27, 24",0.008448
1,2024-02-26,182.24,182.7600,180.65,181.16,181.16,40867421,40867400,-1.08,-0.59263,181.32,"February 26, 24",-0.005926
2,2024-02-23,185.01,185.0400,182.23,182.52,182.52,44926677,45074500,-2.49,-1.35000,182.99,"February 23, 24",-0.013500
3,2024-02-22,183.48,184.9550,182.46,184.37,184.37,52284192,52292200,0.89,0.48507,183.84,"February 22, 24",0.004851
4,2024-02-21,181.94,182.8888,180.66,182.32,182.32,41496371,41529700,0.38,0.20886,181.99,"February 21, 24",0.002089
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504,2018-03-06,44.48,44.5600,44.03,44.17,41.88,95154000,95154000,-0.31,-0.69694,44.25,"March 06, 18",-0.006969
1505,2018-03-05,43.80,44.4400,43.63,44.21,41.92,113605600,113605600,0.41,0.93607,44.09,"March 05, 18",0.009361
1506,2018-03-02,43.20,44.0800,43.11,44.05,41.77,153816000,153816000,0.85,1.97000,43.75,"March 02, 18",0.019700
1507,2018-03-01,44.63,44.9500,43.17,43.75,41.49,195208000,195208000,-0.88,-1.97000,43.95,"March 01, 18",-0.019700


In [4]:
def process_and_upload_ohlc(stock_list):
    ctx = snowflake.connector.connect(
        user=SNF_USER,
        password=SNF_PASSWORD,
        account=SNF_ACCOUNT,
        warehouse=SNF_WAREHOUSE,
        database=SNF_DATABASE,
        schema='PUBLIC'
    )
    cs = ctx.cursor()
    
    # Create the OHLC table
    create_table_query = """
    CREATE OR REPLACE TABLE ohlc_6Y (
        symbol VARCHAR,
        date DATE,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        adjClose FLOAT,
        volume INT,
        unadjustedVolume INT,
        change FLOAT,
        changePercent FLOAT,
        vwap FLOAT,
        label VARCHAR,
        changeOverTime FLOAT,
        SNAPSHOT_DATE DATE
    )
    """
    cs.execute(create_table_query)
    
    # Process each stock
    for symbol in stock_list:
        print(f"Fetching data for {symbol}")
        df = fetch_ohlc_data(symbol)
        if not df.empty:
            df['symbol'] = symbol  # Add symbol column to match table schema
            df.columns = [column.upper() for column in df.columns]  # Adjust as necessary to match Snowflake's schema
            df['DATE'] = pd.to_datetime(df['DATE'],errors='coerce').dt.date
            df['SNAPSHOT_DATE'] = datetime.now().date() 
            write_pandas(ctx, df, 'OHLC_6Y')
            print(f"Uploaded data for {symbol}")

    cs.close()
    ctx.close()




In [5]:
## get master stock list from the financial_ratios table instead of calling the api again 
import snowflake.connector

def fetch_master_stock_list():
    # Snowflake connection details
    ctx = snowflake.connector.connect(
        user=SNF_USER,
        password=SNF_PASSWORD,
        account=SNF_ACCOUNT,
        warehouse=SNF_WAREHOUSE,
        database=SNF_DATABASE,
        schema='PUBLIC'
    )
    
    # Create cursor object
    cs = ctx.cursor()
    
    try:
        # Execute the query to fetch distinct stock symbols
        cs.execute("SELECT DISTINCT symbol FROM stock_analytics.public.financial_ratios")
        
        # Fetch all the results
        result = cs.fetchall()
        
        # Extract symbols from the result set
        master_stock_list = [row[0] for row in result]
        return master_stock_list
        
    except Exception as e:
        print(f"Failed to fetch master stock list: {e}")
        return []
    finally:
        # Close cursor and connection
        cs.close()
        ctx.close()

# Use the fetched master_stock_list for further processing
master_stock_list = fetch_master_stock_list()
print(f"Fetched {len(master_stock_list)} symbols from Snowflake.")


Fetched 599 symbols from Snowflake.


### finally run the calls to the apis for all the symbols in the financial_ratios table

In [6]:
# Example usage with a sample stock list
process_and_upload_ohlc(master_stock_list)

Fetching data for FDS
Uploaded data for FDS
Fetching data for CTAS
Uploaded data for CTAS
Fetching data for WFC
Uploaded data for WFC
Fetching data for NSC
Uploaded data for NSC
Fetching data for BHF
Uploaded data for BHF
Fetching data for CRL
Uploaded data for CRL
Fetching data for KDP
Uploaded data for KDP
Fetching data for HP
Uploaded data for HP
Fetching data for VRSK
Uploaded data for VRSK
Fetching data for FBHS
Uploaded data for FBHS
Fetching data for GNRC
Uploaded data for GNRC
Fetching data for ITW
Uploaded data for ITW
Fetching data for CVS
Uploaded data for CVS
Fetching data for FLIR
Uploaded data for FLIR
Fetching data for SCHW
Uploaded data for SCHW
Fetching data for RF
Uploaded data for RF
Fetching data for FAST
Uploaded data for FAST
Fetching data for TFC
Uploaded data for TFC
Fetching data for MDLZ
Uploaded data for MDLZ
Fetching data for IPG
Uploaded data for IPG
Fetching data for TT
Uploaded data for TT
Fetching data for EXPD
Uploaded data for EXPD
Fetching data for FF